## This notebook does the following:

 <li>"Breaks" the stacks of CIR and RGB</li>
 <li>Gets the bands Red, Green, Blue and Near Infrared and stacks them</li>
 <li>Processes and merges the DSM (in this notebook, not run)</li>

In [1]:
import rasterio as rio
import numpy as np
import geopandas as gpd
import pandas as pd

In [5]:
import rasterio.mask

In [2]:
import os
import glob

## "Break" the multiband .tif data to get single bands <br> We get Near Infrarred from CIR orthophotos and Red, Green and Blue from RGB orthophotos <br> Both areas are processed

In [3]:

folder = r".\raster"
suff = ['cir','rgb']

data_tiff = [ tif for tif in list(glob.glob(os.path.join(folder, "*.tif") )) if any(x in tif for x in suff  )]# This tif has 3 bands and we want to split them


wavel = ["red","green","blue", "nir"] # wavelengths to name the "splitted " bands


for tifs in data_tiff:
    
    print(tifs)
    
    if tifs.endswith("cir.tif"):
        name_tif = tifs.split("_")[0].split("\\")[-1]
        
        with rio.open(tifs) as src:
            out_meta = src.meta.copy()
            
            single_band = src.read(1) # Near infrarred is the first band in the target stack. Get only this one
            
            out_meta.update({"count": 1}) 
            
            out_name = wavel[wavel.index('nir')]+"_"+name_tif+".tif"

            out_img = os.path.join(folder, out_name)
             #save the clipped raster to disk
            with rio.open(out_img, "w", **out_meta) as dest:
                dest.write(single_band,1)
            print("single band: ",out_name)
                 
    else:  
        with rio.open(tifs) as src: 
            i = 0
            out_meta = src.meta.copy()
                   
            for band in range(1, src.count+1):
                single_band = src.read(band) # BANDS ARE ORDERED AS RGB. Get all of them
        
                out_name = wavel[i]+"_"+name_tif+".tif"
                out_img = os.path.join(folder, out_name)

                out_meta.update({"count": 1})

                # save the clipped raster to disk
                with rio.open(out_img, "w", **out_meta) as dest:
                    dest.write(single_band,1)
                print("single band: ",out_name)
                
                i = i +1
        


.\raster\54752_cir.tif
single band:  nir_54752.tif
.\raster\54752_rgb.tif
single band:  red_54752.tif
single band:  green_54752.tif
single band:  blue_54752.tif
.\raster\54761_cir.tif
single band:  nir_54761.tif
.\raster\54761_rgb.tif
single band:  red_54761.tif
single band:  green_54761.tif
single band:  blue_54761.tif


## Clip the single bands according to the study areas

In [8]:

folder = r".\raster"

study_areas = gpd.read_file(r".\vector\Study_areas.shp")
print(study_areas.crs)

preff = [ "nir","red","green","blue"]

data_tiff = [ tif for tif in list(glob.glob(os.path.join(folder, "*.tif") )) if any([tif.split("_")[0].split("\\")[-1].startswith(item) for item in preff])]



for tif in data_tiff:
    
    i = 0
    for area in study_areas['area_n'].unique():
        
        
        if (area == "area_2" or area == "area_1" ) and ( tif.split("\\")[-1].split("_")[1].split(".")[0] == "54752" ):
            
            selection = study_areas.loc[study_areas['area_n'] == area]
            
            with rio.open(tif) as src:
                #clip (mask)
                out_image, out_transform = rio.mask.mask(src,selection['geometry'], all_touched = True, crop=True) ## Check again
                
                out_meta = src.meta
                src.close()
                
            out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

            out_clip = tif.replace(".tif", "_clip_"+area+".tif")
            
            with rio.open(out_clip, "w", **out_meta) as dest:
                dest.write(out_image)
                print(out_clip)
                
        
        elif (area == "area_3" ) and ( tif.split("\\")[-1].split("_")[1].split(".")[0] == "54761" ):
            selection = study_areas.loc[study_areas['area_n'] == area]
            
            with rio.open(tif) as src:
                 #clip (mask)
                out_image, out_transform = rio.mask.mask(src,selection['geometry'], all_touched = True, crop=True) ## Check again

                out_meta = src.meta
                src.close()

            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})

            out_clip = tif.replace(".tif", "_clip_"+area+".tif")

            with rio.open(out_clip, "w", **out_meta) as dest:
                dest.write(out_image)
                print(out_clip)
   

epsg:3301
.\raster\blue_54752_clip_area_1.tif
.\raster\blue_54752_clip_area_2.tif
.\raster\blue_54761_clip_area_3.tif
.\raster\green_54752_clip_area_1.tif
.\raster\green_54752_clip_area_2.tif
.\raster\green_54761_clip_area_3.tif
.\raster\nir_54752_clip_area_1.tif
.\raster\nir_54752_clip_area_2.tif
.\raster\nir_54761_clip_area_3.tif
.\raster\red_54752_clip_area_1.tif
.\raster\red_54752_clip_area_2.tif
.\raster\red_54761_clip_area_3.tif


## STACK the bands to a .tif with 4 bands: NIR - RED - GREEN - BLUE. Decreasing order of wavelength

In [10]:
folder = r".\raster"
preff = [ "nir","red","green","blue"]


# The following list gets bands by ALPHABETICAL order.  
data_tiff = [ tif for tif in list(glob.glob(os.path.join(folder, "*.tif") )) \
             if any([(tif.split("_")[0].split("\\")[-1].startswith(item)) for item in preff]) \
             and ("clip" in tif)]

# Iterate in each area.
for area in study_areas['area_n'].unique():
    
    bands_each_area = [element for element in data_tiff \
                  if element.split(".")[-2][-6:] == area] # Select bands from one specific area to stack them. THIS IS THE MAIN LIST
    
    # select near infrared ONLY and keep it.
    data_tiff_nir = [tif for tif in bands_each_area if ("nir" in tif  )][0]
    
  
    # remove from the MAIN LIST the NIR band
    bands_each_area.remove(data_tiff_nir)

    # inssert in the last place the NIR band. 
    bands_each_area.insert(len(bands_each_area), data_tiff_nir)
    
    #Then , check that all are in correct order
    print(" ".join(bands_each_area))

    
    with rio.open(bands_each_area[1]) as src0:
        meta = src0.meta # get the metadata (profile) of any of the band. In here, just the first one, and close it.
        src0.close()
      # Update meta to reflect the number of layers
    meta.update(count = len(bands_each_area))
    # Read each layer and write it to stack
    
    output_stack = os.path.join(folder,bands_each_area[0][bands_each_area[0].find("_")+1:-4]+"_STK.tif")
    
    with rio.open(output_stack, 'w', **meta) as dst:
        for id, layer in enumerate(bands_each_area, start=1):
            with rio.open(layer) as src1:
                dst.write_band(id, src1.read(1))
        print("band",output_stack, "saved")
        print("--")
        dst.close()
        
print("FINISHED------")


.\raster\blue_54752_clip_area_1.tif .\raster\green_54752_clip_area_1.tif .\raster\red_54752_clip_area_1.tif .\raster\nir_54752_clip_area_1.tif
band .\raster\54752_clip_area_1_STK.tif saved
--
.\raster\blue_54752_clip_area_2.tif .\raster\green_54752_clip_area_2.tif .\raster\red_54752_clip_area_2.tif .\raster\nir_54752_clip_area_2.tif
band .\raster\54752_clip_area_2_STK.tif saved
--
.\raster\blue_54761_clip_area_3.tif .\raster\green_54761_clip_area_3.tif .\raster\red_54761_clip_area_3.tif .\raster\nir_54761_clip_area_3.tif
band .\raster\54761_clip_area_3_STK.tif saved
--
FINISHED------


## Process Digital Surface models within the study areas

In [ ]:
#downloaded digital surface model DSM resolution 1m
#GeoTIFFhttps://geoportaal.maaamet.ee/eng/Maps-and-Data/Elevation-data/Download-Elevation-Data-p664.html

#our areas arent coverd by 1 DSM so had to download 10 DSM images
tiffs_path = ['473658_ndsm_abs_1m.tif','474658_ndsm_abs_1m.tif', '474659_ndsm_abs_1m.tif', '472657_ndsm_abs_1m.tif', #4 tifs that cover area 1
              '471658_ndsm_abs_1m.tif','471657_ndsm_abs_1m.tif', '472658_ndsm_abs_1m.tif', '473659_ndsm_abs_1m.tif', #4 tifs that cover area 2
             '474662_ndsm_abs_1m.tif', '473662_ndsm_abs_1m.tif' #2 tifs that cover area 3
              ]

tiffs_path = [s + ".\\raster\\\dsm\\" for s in tiffs_path]

raster_to_mosaic = []

for x in tiffs_path:
    raster = rasterio.open(x)
    raster_to_mosaic.append(raster)

mosaic, output = merge(raster_to_mosaic) #merging all the listed DSM tifs together to create one

output_meta = raster.meta.copy()
output_meta.update({
        'driver' :'GTiff',
        'height': mosaic.shape[1],
        'width': mosaic.shape[2],
        'crs':'EPSG:3301',
        'compress':'lzw',
        'transform': output
        })

with rasterio.open('.\\raster\\dsm\\merged_DSM.tiff', 'w+', **output_meta) as m:
        m.write(mosaic)

In [ ]:
#looking at the merged DSM
dsm = rasterio.open('merged_DSM.tiff')
show(dsm)
print(dsm.crs) # Check the coordinate projection of the DSM

## clip the digital surface models to the sutdy areas

In [ ]:
#code help from:
#https://kodu.ut.ee/~kmoch/geopython2022/L5/raster.html#clipping-a-raster

#clipping the merged DSM with study areas

for area in study_areas.area_n.unique():


    selection = study_areas.loc[study_areas["area_n"] == area]

    out_image, out_transform = mask(dataset=dsm, shapes=selection['geometry'], crop=True) #masking the merged DSM with area 1 polygon
    out_meta = dsm.meta.copy() #copyng the metadata from the merged DSM

    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
    #writing out area DSM
    with rasterio.open(".\\raster\\dsm"+area+"_dsm.tif", "w", **out_meta) as dest: 
        dest.write(out_image)


## Check the DSMs

In [ ]:
#looking at the results
#area 1
dsm_area_1 = rasterio.open(".\\raster\\dsm\\area_1_dsm.tif")
show(dsm_area_1)
print(dsm_area_1.crs) #cheking that we have the correct prjection (crs = 3301)

#area 2
dsm_area_2 = rasterio.open(".\\raster\\dsm\\area_2_dsm.tif")
show(dsm_area_2)
print(dsm_area_2.crs)

#area 3
dsm_area_3 = rasterio.open(".\\raster\\dsm\\area_3_dsm.tif")
show(dsm_area_3)
print(dsm_area_3.crs)